In [1]:
import pandas as pd
import pymysql as mc
import os
import sys
import time
import numpy as np
import getpass

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
pw = getpass.getpass()
cnx = mc.connect(user='akashgoyal',password=pw,
                 host='stromberg.cs.uchicago.edu',db='mlb_practicum',port=3306)

········


In [46]:
def reclassify_pitches2(df):
        four_seam = ['FF','FA']
        slider = ['SL']
        two_seam = ['FT']
        change_up = ['CH']
        sinker = ['SI','FS']
        knuckleball = ['KN']
        curve = ['CU','KC']
        cutter = ['FC']
        df['pitchType'][df['pitchType'].isin(four_seam)] = 'four_seam'
        df['pitchType'][df['pitchType'].isin(slider)] = 'slider'
        df['pitchType'][df['pitchType'].isin(two_seam)] = 'two_seam'
        df['pitchType'][df['pitchType'].isin(change_up)] = 'change_up'
        df['pitchType'][df['pitchType'].isin(sinker)] = 'sinker'
        df['pitchType'][df['pitchType'].isin(knuckleball)] = 'knuckleball'
        df['pitchType'][df['pitchType'].isin(curve)] = 'curve'
        df['pitchType'][df['pitchType'].isin(cutter)] = 'cutter'
        return df

class slugging_pct:
    def __init__(self,df,trailing_ab,min_abs):
        self.df = df
        self.min_abs = min_abs
        self.trailing_ab = trailing_ab
        self.mapped_df = self.aggregation_helper(df)


    def aggregation_helper(self,df):
        possible_pitches = ['four_seam','slider','two_seam','change_up','sinker',
                        'knuckleball','curve','cutter']

        grouped = df.groupby(['gameID','abNum']).tail(1) #this gets the action pitch of the at bat
        grouped = grouped[grouped['pitchType'].notnull()] #get rid of pitches which we dont know the type
        ball_in_play = ['In play, out(s)','In play, run(s)','In play, no out'] #valid balls in play
        grouped = grouped[grouped.pitchDes.isin(ball_in_play)] #only get balls in play

     #create a dictionary mapping bases to each event
        ball_in_play_outs = ['Groundout','Flyout','Pop Out','Lineout','Sac Fly',
                        'Grounded Into DP','Forceout','Double Play', 'Field Error',
                        'Bunt Groundout','Sac Bunt','Fielders Choice Out','Bunt Pop Out',
                        'Fielders Choice','Bunt Lineout','Sac Fly DP','Triple Play',
                        'Sacrifice Bunt DP','Batter Interference','Fan interference',
                        'Catcher Interference']
        in_play_dict = {i:0 for i in ball_in_play_outs}
        in_play_dict['Home Run'] = 4
        in_play_dict['Triple'] = 3
        in_play_dict['Double'] = 2
        in_play_dict['Single'] = 1
        grouped['bases'] = grouped['the_event'].map(in_play_dict)
        return grouped
    
    def create_pivot(self,gameDate,df):
        grouped = df.groupby(['pitcherID']).tail(self.trailing_ab)

        def pivot_helper(df):
            total = df.sum()
            if len(df)>self.min_abs:
                return total/len(df)
            return '\\N'

        #now calculate the slugging percent
        pivot_table = pd.pivot_table(grouped,values=['bases'],columns=['pitchType'],
                                     index=['pitcherID'], aggfunc=pivot_helper,
                                    fill_value='\\N')

        pivot_table['gameDate'] = gameDate
        return pivot_table

In [42]:
query = """SELECT gameDate,Pitch2.gameID AS gameID, abNum,pitcherID, batterHand,
        the_event,pitchID, pitchType,pitchDes,zone FROM Pitch2 INNER JOIN Game
        ON Game.gameID = Pitch2.gameID WHERE ((pitchType
        NOT IN ('IN','PO','FO','EP','SC','UN','AB')) OR pitchType is NULL)"""

df = pd.read_sql_query(query,cnx)

In [43]:
df2 = df.copy()
df2 = reclassify_pitches2(df2)
df2 = df2.sort_values(['gameDate','gameID','pitchID'],ascending=[True,True,True])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be s

In [44]:
a = slugging_pct(df2,500,20)
distinct_dates = df2.gameDate.unique()
full_pivot_table = 'FILLER'
start_type = type(full_pivot_table)

for i,day in enumerate(distinct_dates):
        df = a.mapped_df[a.mapped_df['gameDate']<day]
        if len(df)==0:
                continue

        piv = a.create_pivot(day,df)
        if type(full_pivot_table)!=start_type and len(full_pivot_table)>0:
                full_pivot_table = full_pivot_table.append(piv)

        elif len(piv)!=0:
                full_pivot_table = piv

        if i % 2 == 0:
                print (i,day)
#full_pivot_table.to_csv('pitcher slugging_percentages.txt')

2 2010-04-06
4 2010-04-08
6 2010-04-10
8 2010-04-12
10 2010-04-14
12 2010-04-16
14 2010-04-18
16 2010-04-20
18 2010-04-22
20 2010-04-24
22 2010-04-26
24 2010-04-28
26 2010-04-30
28 2010-05-02
30 2010-05-04
32 2010-05-06
34 2010-05-08
36 2010-05-10
38 2010-05-12
40 2010-05-14
42 2010-05-16
44 2010-05-18
46 2010-05-20
48 2010-05-22
50 2010-05-24
52 2010-05-26
54 2010-05-28
56 2010-05-30
58 2010-06-01
60 2010-06-03
62 2010-06-05
64 2010-06-07
66 2010-06-09
68 2010-06-11
70 2010-06-13
72 2010-06-15
74 2010-06-17
76 2010-06-19
78 2010-06-21
80 2010-06-23
82 2010-06-25
84 2010-06-27
86 2010-06-29
88 2010-07-01
90 2010-07-03
92 2010-07-05
94 2010-07-07
96 2010-07-09
98 2010-07-11
100 2010-07-16
102 2010-07-18
104 2010-07-20
106 2010-07-22
108 2010-07-24
110 2010-07-26
112 2010-07-28
114 2010-07-30
116 2010-08-01
118 2010-08-03
120 2010-08-05
122 2010-08-07
124 2010-08-09
126 2010-08-11
128 2010-08-13
130 2010-08-15
132 2010-08-17
134 2010-08-19
136 2010-08-21
138 2010-08-23
140 2010-08-25
142

1096 2016-04-19
1098 2016-04-21
1100 2016-04-23
1102 2016-04-25
1104 2016-04-27
1106 2016-04-29
1108 2016-05-01
1110 2016-05-03
1112 2016-05-05
1114 2016-05-07
1116 2016-05-09
1118 2016-05-11
1120 2016-05-13
1122 2016-05-15
1124 2016-05-17
1126 2016-05-19
1128 2016-05-21
1130 2016-05-23
1132 2016-05-25
1134 2016-05-27
1136 2016-05-29
1138 2016-05-31
1140 2016-06-02
1142 2016-06-04
1144 2016-06-06
1146 2016-06-08
1148 2016-06-10
1150 2016-06-12
1152 2016-06-14
1154 2016-06-16
1156 2016-06-18
1158 2016-06-20
1160 2016-06-22
1162 2016-06-24
1164 2016-06-26
1166 2016-06-28
1168 2016-06-30
1170 2016-07-02
1172 2016-07-04
1174 2016-07-06
1176 2016-07-08
1178 2016-07-10
1180 2016-07-16
1182 2016-07-18
1184 2016-07-20
1186 2016-07-22
1188 2016-07-24
1190 2016-07-26
1192 2016-07-28
1194 2016-07-30
1196 2016-08-01
1198 2016-08-03
1200 2016-08-05
1202 2016-08-07
1204 2016-08-09
1206 2016-08-11
1208 2016-08-13
1210 2016-08-15
1212 2016-08-17
1214 2016-08-19
1216 2016-08-21
1218 2016-08-23
1220 201

In [45]:
full_pivot_table

bases                                         \
pitchType            change_up     curve cutter four_seam knuckleball   
pitcherID batterHand                                                    
120221    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
150011    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
277417    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
282332    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
430673    R                 \N        \N     \N        \N         NaN   
449097    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
453268    L                 \N        \N     \N        \N         NaN   
501955    R                 \N        \N     \N        \N         NaN   
502085    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
506606    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
110683    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
112020    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
117955    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
119984    R                 \N        \N     \N        \N         NaN   
120221    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
123790    R                 \N        \N     \N        \N         NaN   
133460    L                 \N        \N     \N        \N         NaN   
          R                 \N        \N     \N        \N         NaN   
133982    L                 \N        \N     \N        \N         NaN   
...                        ...       ...    ...       ...         ...   
656427    L                 \N        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
656546    L            0.54902  0.409091     \N  0.520661          \N   
          R                 \N   0.52381     \N  0.692857          \N   
656547    L                 \N        \N     \N        \N          \N   
          R                 \N  0.444444     \N  0.567568          \N   
656756    L                 \N        \N     \N        \N          \N   
          R           0.354167  0.603175     \N  0.534247          \N   
656794    L                 \N  0.458333     \N  0.586957          \N   
          R           0.638889  0.388889     \N  0.606061          \N   
656945    L                 \N        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
657205    L                 \N        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
657610    L            0.52381        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
657670    L                 \N        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
657681    L                 \N        \N     \N        \N          \N   
          R                 \N        \N     \N        \N          \N   
658792    L                 \N        \N     \N        \N          \N   
          

In [47]:
df3 = df.copy()
df3 = reclassify_pitches2(df3)
b = slugging_pct(df3,500,20)
distinct_dates = df3.gameDate.unique()
full_pivot_table2 = 'FILLER'
start_type = type(full_pivot_table2)

for i,day in enumerate(distinct_dates):
        df = b.mapped_df[b.mapped_df['gameDate']<day]
        if len(df)==0:
                continue

        piv = b.create_pivot(day,df)
        if type(full_pivot_table2)!=start_type and len(full_pivot_table2)>0:
                full_pivot_table2 = full_pivot_table2.append(piv)

        elif len(piv)!=0:
                full_pivot_table2 = piv

        if i % 2 == 0:
                print (i,day)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be s

2 2010-04-06
4 2010-04-08
6 2010-04-10
8 2010-04-12
10 2010-04-14
12 2010-04-16
14 2010-04-18
16 2010-04-20
18 2010-04-22
20 2010-04-24
22 2010-04-26
24 2010-04-28
26 2010-04-30
28 2010-05-02
30 2010-05-04
32 2010-05-06
34 2010-05-08
36 2010-05-10
38 2010-05-12
40 2010-05-14
42 2010-05-16
44 2010-05-18
46 2010-05-20
48 2010-05-22
50 2010-05-24
52 2010-05-26
54 2010-05-28
56 2010-05-30
58 2010-06-01
60 2010-06-03
62 2010-06-05
64 2010-06-07
66 2010-06-09
68 2010-06-11
70 2010-06-13
72 2010-06-15
74 2010-06-17
76 2010-06-19
78 2010-06-21
80 2010-06-23
82 2010-06-25
84 2010-06-27
86 2010-06-29
88 2010-07-01
90 2010-07-03
92 2010-07-05
94 2010-07-07
96 2010-07-09
98 2010-07-11
100 2010-07-16
102 2010-07-18
104 2010-07-20
106 2010-07-22
108 2010-07-24
110 2010-07-26
112 2010-07-28
114 2010-07-30
116 2010-08-01
118 2010-08-03
120 2010-08-05
122 2010-08-07
124 2010-08-09
126 2010-08-11
128 2010-08-13
130 2010-08-15
132 2010-08-17
134 2010-08-19
136 2010-08-21
138 2010-08-23
140 2010-08-25
142

1096 2016-04-23
1098 2016-04-25
1100 2016-04-27
1102 2016-04-29
1104 2016-05-01
1106 2016-05-03
1108 2016-05-05
1110 2016-05-07
1112 2016-05-09
1114 2016-05-11
1116 2016-05-13
1118 2016-05-15
1120 2016-05-17
1122 2016-05-19
1124 2016-05-21
1126 2016-05-23
1128 2016-05-25
1130 2016-05-27
1132 2016-05-29
1134 2016-05-31
1136 2016-06-02
1138 2016-06-04
1140 2016-06-06
1142 2016-06-08
1144 2016-06-10
1146 2016-06-12
1148 2016-06-14
1150 2016-06-16
1152 2016-06-18
1154 2016-06-20
1156 2016-06-22
1158 2016-06-24
1160 2016-06-26
1162 2016-06-28
1164 2016-06-30
1166 2016-07-02
1168 2016-07-04
1170 2016-07-06
1172 2016-07-08
1174 2016-07-10
1176 2016-07-16
1178 2016-07-18
1180 2016-07-20
1182 2016-07-22
1184 2016-07-24
1186 2016-07-26
1188 2016-07-28
1190 2016-07-30
1192 2016-08-01
1194 2016-08-03
1196 2016-08-05
1198 2016-08-07
1200 2016-08-09
1202 2016-08-11
1204 2016-08-13
1206 2016-08-15
1208 2016-08-17
1210 2016-08-19
1212 2016-08-21
1214 2016-08-23
1216 2016-08-25
1218 2016-08-27
1220 201

In [55]:
df.head()
#df4 = df.copy()
#df4 = reclassify_pitches2(df4)
#df4.head()

,gameDate,gameID,abNum,pitcherID,batterHand,the_event,pitchID,pitchType,pitchDes,zone,bases
751814,2010-04-04,BOS201004040,1,277417,R,Groundout,3,two_seam,"In play, out(s)",2,0
751837,2010-04-04,BOS201004040,7,277417,R,Groundout,46,two_seam,"In play, out(s)",6,0
751839,2010-04-04,BOS201004040,8,277417,L,Lineout,51,four_seam,"In play, out(s)",9,0
751843,2010-04-04,BOS201004040,9,277417,L,Home Run,58,two_seam,"In play, run(s)",2,4
751852,2010-04-04,BOS201004040,11,277417,L,Single,75,four_seam,"In play, no out",11,1


In [ ]:
qry3 = """SELECT YEAR(gameDate), COUNT(*),akash
          FROM Pitch2 INNER JOIN Game ON Game.gameDate = Pitch2.gameDate
          INNER JOIN Pitch_type_mapping ptm ON ptm.original = Pitch2.pitchType
          GROUP BY YEAR(gameDate),akash"""

my_df = pd.read_sql_query(qry3,cnx)